# HE Slide Analysis: Muscle Cell and Nuclei Segmentation

This notebook performs two main tasks on HE stained slide images:
1.  **Muscle Cell Segmentation:** Identifies individual muscle cells (pink/red cytoplasm) using Cellpose.
2.  **Nuclei Segmentation:** Identifies brown nuclei within the cells using Color Deconvolution followed by Cellpose.

## 1. Setup and Imports

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import tifffile
from skimage import io, color, restoration
from cellpose import models
from cellpose import plot
import napari
import os
import torch

# Check for GPU
use_gpu = torch.cuda.is_available()
print(f"GPU Available: {use_gpu}")

GPU Available: True


In [2]:
# Load Image(s) - Supports JPG, PNG, TIF
# Option 1: Process a single image (uncomment to use)
# IMAGE_PATH = r"S:\micro\ts2625\eh2888\lem\HEImages_0123\quantification\01_01_02_0.jpg"

# Option 2: Process all JPG files in a folder (recommended for batch processing)
from pathlib import Path

# Set your folder path here
FOLDER_PATH = r"S:\micro\ts2625\eh2888\lem\HEImages_0123\newData\new"

# Get all JPG files
jpg_files = sorted(Path(FOLDER_PATH).glob("*.jpg"))

print(f"Found {len(jpg_files)} JPG files in {FOLDER_PATH}")
for i, f in enumerate(jpg_files, 1):
    print(f"  {i}. {f.name}")

# Select first image for preview (or change index to select a different one)
if jpg_files:
    IMAGE_PATH = str(jpg_files[0])
    print(f"\nLoading first image for preview: {Path(IMAGE_PATH).name}")
else:
    print("No JPG files found!")
    IMAGE_PATH = None



Found 2 JPG files in S:\micro\ts2625\eh2888\lem\HEImages_0123\newData\new
  1. 01_02_03_1.jpg
  2. 01_08_05_1.jpg

Loading first image for preview: 01_02_03_1.jpg


## Part 1: Muscle Cell Segmentation (Cellpose)

We use Cellpose to segment the muscle cells. 
*   **Model:** `cyto2` (works well for cytoplasm and general cells).
*   **Target:** Red/Pink areas.

In [3]:
# 1. Muscle Segmentation
# Set expected diameter (in pixels) to speed up processing
# Set to None for auto-estimation (slower)

model_cyto = models.CellposeModel(gpu=use_gpu, model_type='cyto3')



# Quick verify

# View with napari

model_type argument is not used in v4.0.1+. Ignoring this argument...


## Part 2: Prepare for ImageJ/FIJI Integration

Create segmentation overlays and ImageJ-compatible files for viewing results in FIJI.

In [4]:
# Process All JPG Files in Batch
# This cell processes all JPG files found in the folder

if jpg_files:
    print(f"\n{'='*60}")
    print(f"BATCH PROCESSING: {len(jpg_files)} JPG files")
    print(f"{'='*60}\n")
    
    results_summary = []
    
    for file_idx, image_file in enumerate(jpg_files, 1):
        IMAGE_PATH = str(image_file)
        filename_base = image_file.stem
        output_dir = image_file.parent
        
        print(f"\n[{file_idx}/{len(jpg_files)}] Processing: {filename_base}")
        print("-" * 60)
        
        try:
            # Load image
            if IMAGE_PATH.lower().endswith(('.tif', '.tiff')):
                img_rgb = tifffile.imread(IMAGE_PATH)
            else:
                img_rgb = io.imread(IMAGE_PATH)
            
            # Channel corrections
            if img_rgb.ndim == 3 and img_rgb.shape[0] < 5: 
                img_rgb = np.transpose(img_rgb, (1, 2, 0))
            if img_rgb.shape[-1] == 4: 
                img_rgb = img_rgb[..., :3]
            
            print(f"  ✓ Loaded | Shape: {img_rgb.shape}")
            
            # Muscle Segmentation
            print(f"  → Segmenting muscle cells...")
            masks_cells, flows, styles = model_cyto.eval(
                img_rgb, 
                diameter=120, 
                flow_threshold=0.3, 
                cellprob_threshold=-1,
            )
            num_cells = masks_cells.max()
            print(f"  ✓ Found {num_cells} muscle cells")
            
            # Create eroded mask and overlay
            from scipy import ndimage
            import cv2
            
            eroded_mask = masks_cells.copy()
            for label_id in np.unique(masks_cells):
                if label_id == 0:
                    continue
                mask_region = (masks_cells == label_id).astype(np.uint8)
                eroded_region = ndimage.binary_erosion(mask_region, iterations=4).astype(np.uint8)
                eroded_mask[masks_cells == label_id] = 0
                eroded_mask[eroded_region > 0] = label_id
            
            # Create overlay
            overlay_cells = img_rgb.copy().astype(np.uint8)
            unique_labels = np.unique(eroded_mask)
            unique_labels = unique_labels[unique_labels != 0]
            
            colors = []
            for i in range(len(unique_labels)):
                hue = int((i / max(len(unique_labels), 1)) * 180)
                hsv_color = np.uint8([[[hue, 255, 255]]])
                bgr_color = cv2.cvtColor(hsv_color, cv2.COLOR_HSV2BGR)[0, 0]
                colors.append(tuple(int(c) for c in bgr_color))
            
            for idx, label_id in enumerate(unique_labels):
                mask_region = (eroded_mask == label_id).astype(np.uint8)
                contours, _ = cv2.findContours(mask_region, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
                if contours:
                    cv2.drawContours(overlay_cells, contours, -1, colors[idx], 1)
            
            # Save outputs
            overlay_path = output_dir / f'{filename_base}_w_bound.tif'
            mask_path = output_dir / f'{filename_base}_mask.tif'
            
            tifffile.imwrite(str(overlay_path), overlay_cells)
            tifffile.imwrite(str(mask_path), eroded_mask.astype(np.uint16))
            
            print(f"  ✓ Saved: {filename_base}_w_bound.tif")
            print(f"  ✓ Saved: {filename_base}_mask.tif")
            
            results_summary.append({
                'file': filename_base,
                'status': '✓ SUCCESS',
                'cells_found': num_cells
            })
            
        except Exception as e:
            print(f"  ✗ ERROR: {str(e)}")
            results_summary.append({
                'file': filename_base,
                'status': f'✗ FAILED: {str(e)}',
                'cells_found': 0
            })
    
    # Print summary
    print(f"\n{'='*60}")
    print("BATCH PROCESSING SUMMARY")
    print(f"{'='*60}")
    for result in results_summary:
        status_str = result['status']
        cells_str = f" | Cells: {result['cells_found']}" if '✓' in status_str else ""
        print(f"{result['file']}: {status_str}{cells_str}")
    
    successful = sum(1 for r in results_summary if '✓' in r['status'])
    print(f"\nCompleted: {successful}/{len(jpg_files)} files processed successfully")
else:
    print("No JPG files to process. Check FOLDER_PATH setting.")



BATCH PROCESSING: 2 JPG files


[1/2] Processing: 01_02_03_1
------------------------------------------------------------
  ✓ Loaded | Shape: (4332, 4230, 3)
  → Segmenting muscle cells...


Resizing is deprecated in v4.0.1+


  ✓ Found 187 muscle cells
  ✓ Saved: 01_02_03_1_w_bound.tif
  ✓ Saved: 01_02_03_1_mask.tif

[2/2] Processing: 01_08_05_1
------------------------------------------------------------
  ✓ Loaded | Shape: (5292, 5016, 3)
  → Segmenting muscle cells...


Resizing is deprecated in v4.0.1+


  ✓ Found 237 muscle cells
  ✓ Saved: 01_08_05_1_w_bound.tif
  ✓ Saved: 01_08_05_1_mask.tif

BATCH PROCESSING SUMMARY
01_02_03_1: ✓ SUCCESS | Cells: 187
01_08_05_1: ✓ SUCCESS | Cells: 237

Completed: 2/2 files processed successfully
